In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
# Getting the list of patients with age more tahn 18 years
patients_adm_age = pd.read_csv('/Adult_patientsAge.csv')

# Getting list of all items along with the source and label
item_id_dbsource = pd.read_csv('./d_items_chartevents.csv')
# Getting list of all items along with the source and label
item_id_labs = pd.read_csv('./d_items_labevents.csv')

In [ ]:
patients_adm_age.head()

In [25]:
patients_adm_age.shape

(50765, 5)

In [ ]:
patients_adm_age[patients_adm_age['age']>=300]

In [27]:
# Whenever there is value >=300 in age column, replace it by 90
patients_adm_age.loc[patients_adm_age['age'] >= 300, 'age'] = 90

In [ ]:
# checking the unique age values
patients_adm_age['age'].unique()

In [29]:
list_itemid_num_MV = item_id_dbsource[(item_id_dbsource['dbsource']=='metavision') & (item_id_dbsource['param_type'].isin(['Numeric','Numeric with tag']))]['itemid']
list_itemid_cat_MV = item_id_dbsource[(item_id_dbsource['dbsource']=='metavision') & (item_id_dbsource['param_type'].isin(['Checkbox']))]['itemid']

In [30]:
print( "num of numerics in MV ", len(list_itemid_num_MV))
print( "num of cat in MV ", len(list_itemid_cat_MV))

num of numerics in MV  617
num of cat in MV  307


## Processing Labs

In [8]:
Mimic_lab_MV = pd.read_csv('mimic_labevents_metavision.csv')
Mimic_lab_CV = pd.read_csv('mimic_labevents_carevue.csv')

In [ ]:
Mimic_lab_CV.head()

In [10]:
Mimic_lab_MV_Formatted_value = Mimic_lab_MV.pivot(index='hadm_id', columns='itemid', values='valuenum')
Mimic_lab_CV_Formatted_value = Mimic_lab_CV.pivot(index='hadm_id', columns='itemid', values='valuenum')

In [11]:
Mimic_lab_CV_Formatted_value.columns

Int64Index([50800, 50801, 50802, 50803, 50804, 50805, 50806, 50808, 50809,
            50810,
            ...
            51511, 51512, 51513, 51514, 51515, 51516, 51517, 51518, 51519,
            51523],
           dtype='int64', name='itemid', length=581)

In [12]:
Mimic_lab_MV_Formatted_value.columns

Int64Index([50800, 50801, 50802, 50803, 50804, 50805, 50806, 50808, 50809,
            50810,
            ...
            51512, 51513, 51514, 51515, 51516, 51517, 51518, 51519, 51523,
            51529],
           dtype='int64', name='itemid', length=576)

In [13]:
# Selecting only labs that are common between two eras becasue the labs are going to be treated as known mapped features
Commonlab_lists =list(set(Mimic_lab_MV_Formatted_value.columns).intersection(set(Mimic_lab_CV_Formatted_value.columns)))
Mimic_lab_CV_Formatted_value= Mimic_lab_CV_Formatted_value[Commonlab_lists]

In [14]:
Mimic_lab_MV_Formatted_value=Mimic_lab_MV_Formatted_value[Commonlab_lists]

In [ ]:
# Computing missing percentage and variance of each lab_item
percent_missing_CV_lab = Mimic_lab_CV_Formatted_value.isnull().sum() *100/ len(Mimic_lab_CV_Formatted_value)
Variance_CV_lab = Mimic_lab_CV_Formatted_value.var().round(2)
missing_value_df_CV_lab = pd.DataFrame({'column_name': Mimic_lab_CV_Formatted_value.columns,
                                 'percent_missing_CV': percent_missing_CV_lab, 'variance_CV' : Variance_CV_lab})
missing_value_df_CV_lab.reset_index(inplace=True)
missing_value_df_CV_lab = pd.merge(item_id_labs, missing_value_df_CV_lab, how="inner", on=["itemid"])
missing_value_df_CV_lab.sort_values(by='itemid', inplace=True)
missing_value_df_CV_lab.set_index('itemid', inplace=True)
missing_value_df_CV_lab.drop(columns=['row_id', 'column_name', 'fluid', 'category','loinc_code'], inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_df_CV_lab.round(3))
    
# filtering out the columns by missingness percent 
# missing_value_df_CV_lab[missing_value_df_CV_lab['percent_missing_CV']<40]

In [ ]:
# Computing missing percentage and variance of each lab_item
percent_missing_MV_lab = Mimic_lab_MV_Formatted_value.isnull().sum() *100/ len(Mimic_lab_MV_Formatted_value)
Variance_MV_lab = Mimic_lab_MV_Formatted_value.var().round(2)
missing_value_df_MV_lab = pd.DataFrame({'column_name': Mimic_lab_MV_Formatted_value.columns,
                                 'percent_missing_MV': percent_missing_MV_lab, 'variance_MV' : Variance_MV_lab})
missing_value_df_MV_lab.reset_index(inplace=True)
missing_value_df_MV_lab = pd.merge(item_id_labs, missing_value_df_MV_lab, how="inner", on=["itemid"])
missing_value_df_MV_lab.sort_values(by='itemid', inplace=True)
missing_value_df_MV_lab.set_index('itemid', inplace=True)
missing_value_df_MV_lab.drop(columns=['row_id', 'column_name', 'fluid', 'category','loinc_code'], inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_df_MV_lab.round(3))
    
# filtering out the columns by missingness percent 
# missing_value_df_CV_lab[missing_value_df_CV_lab['percent_missing_CV']<40]

In [ ]:
missing_value_lab_both = pd.concat([missing_value_df_CV_lab, missing_value_df_MV_lab[['percent_missing_MV','variance_MV']]], axis=1)
missing_value_lab_both.sort_values(by='percent_missing_CV', ascending=False, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_lab_both)

In [18]:
# reseting index for plotting
missing_value_lab_both.reset_index(inplace=True)

In [ ]:
# # plotting the missingness rate and variance
missing_value_lab_both.plot(x='label', y=["percent_missing_CV", "percent_missing_MV"])
plt.title("Missingness percentage")
plt.show()
plt.close()

# missing_value_lab_both.plot(x='label', y=["percent_missing_CV", "percent_missing_MV"])
missing_value_lab_both.plot(x='label', y=["variance_CV", "variance_MV"], rot=0)
# ax = missing_value_lab_both.plot(
#     x='label', y=["percent_missing_CV", "percent_missing_MV"], linestyle='-', marker='o')
# missing_value_lab_both.plot(x='label', y=["variance_CV", "variance_MV"], kind='bar',ax=ax,  rot=0)
plt.title("Variance")
# plt.title("Missingness percentage, Variance")
plt.yscale('linear')
plt.show()
plt.close()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_lab_both.sort_values(by='itemid', ascending=True))

In [33]:
# Selecting only those that have missing value less than approximately 90 percent (58 in number)
Final_lab_itemids_used = list(missing_value_lab_both[(missing_value_lab_both['percent_missing_CV']<90) & (missing_value_lab_both['percent_missing_MV']<90)].index)

In [34]:
# Final lab dataset
Mimic_lab_CV_Formatted_value= Mimic_lab_CV_Formatted_value[Final_lab_itemids_used]
Mimic_lab_MV_Formatted_value=Mimic_lab_MV_Formatted_value[Final_lab_itemids_used]

In [35]:
# Lab data observations rows 
print('Care vue era lab admissions * labitems : ', Mimic_lab_CV_Formatted_value.shape)
print('Meta vision era lab admissions * labitems : ', Mimic_lab_MV_Formatted_value.shape)

Care vue era lab admissions * labitems :  (32156, 58)
Meta vision era lab admissions * labitems :  (21295, 58)


In [ ]:
Mimic_lab_CV_Formatted_value.head()

In [ ]:
Mimic_lab_MV_Formatted_value.head()

In [ ]:
# checking dtypes of the columns
Mimic_lab_MV_Formatted_value.dtypes

## Processing chartevents

In [3]:
Mimic_chart_MV = pd.read_csv('mimic_chartevents_metavision.csv')
Mimic_chart_CV = pd.read_csv('mimic_chartevents_carevue.csv')

/home/trips/Virtual_env_python/lib64/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# These two icustays are extra for one hadm_id. Not sure how they got in.
Mimic_chart_MV = Mimic_chart_MV[Mimic_chart_MV['icustay_id'] != 281378]
Mimic_chart_CV = Mimic_chart_CV[Mimic_chart_CV['icustay_id'] != 244034] 

In [5]:
# It is known that error = 1 in Metavision had measurement errors so selecting observations containing only error=0 cases
Mimic_chart_MV = Mimic_chart_MV[Mimic_chart_MV['error']==0]

In [6]:
print("CV orig obs shape ",Mimic_chart_CV.shape)
print("MV orig obs shape ",Mimic_chart_MV.shape)

CV orig obs shape  (5728193, 17)
MV orig obs shape  (5089801, 17)


In [ ]:
Mimic_chart_CV.head()

In [ ]:
Mimic_chart_MV.head()

In [ ]:
Mimic_chart_CV.dtypes

In [9]:
# Changing the data view to get it in the subjects in rows and items in columns
Mimic_chart_MV_Formatted_value = Mimic_chart_MV.pivot(index='hadm_id', columns='itemid', values='value')
Mimic_chart_CV_Formatted_value = Mimic_chart_CV.pivot(index='hadm_id', columns='itemid', values='value')

In [ ]:
Mimic_chart_CV_Formatted_value

In [ ]:
Mimic_chart_MV_Formatted_value.head()

In [ ]:
# Getting the item ids that are metavision but used in carevue icustays
Common_Chart_list = list(set(Mimic_chart_MV_Formatted_value.columns).intersection(set(Mimic_chart_CV_Formatted_value.columns)))

In [40]:
# dropping the extra columns/itemids from metavision that appeared in carevue data
# Removing only from CV dataset (and not from MV dataset) because these columns originaly beong to Metavision
Mimic_chart_CV_Formatted_value.drop(columns=Common_Chart_list, inplace=True)

In [41]:
# check to make sure if there is any common itemid between two eras
if list(set(Mimic_chart_MV_Formatted_value.columns).intersection(set(Mimic_chart_CV_Formatted_value.columns))) ==[]:
    print(" No common itemid. All good here!")

 No common itemid. All good here!


In [42]:
# check to make sure if there is any common admission between two eras
if list(set(Mimic_chart_MV_Formatted_value.index).intersection(set(Mimic_chart_CV_Formatted_value.index))) ==[]:
        print(" No common hadm_id. All good here!")

 No common hadm_id. All good here!


In [43]:
print("CV formatted obs shape ",Mimic_chart_CV_Formatted_value.shape)
print("MV formatted obs shape ",Mimic_chart_MV_Formatted_value.shape)

CV formatted obs shape  (33669, 2751)
MV formatted obs shape  (21429, 1538)


In [44]:
# Selecting hadm_ids associate with patients older than 18 years
adult_hadmid_CV = list(set(Mimic_chart_CV_Formatted_value.index).intersection(set(list(patients_adm_age['hadm_id']))))
adult_hadmid_MV = list(set(Mimic_chart_MV_Formatted_value.index).intersection(set(list(patients_adm_age['hadm_id']))))

In [45]:
# selecting only adult patients in the dataset
Mimic_chart_CV_Formatted_value = Mimic_chart_CV_Formatted_value.loc[adult_hadmid_CV]
Mimic_chart_MV_Formatted_value = Mimic_chart_MV_Formatted_value.loc[adult_hadmid_MV]

In [46]:
print("CV formatted (ADULTS) obs shape ",Mimic_chart_CV_Formatted_value.shape)
print("MV formatted (ADULTS) obs shape ",Mimic_chart_MV_Formatted_value.shape)

CV formatted (ADULTS) obs shape  (26383, 2751)
MV formatted (ADULTS) obs shape  (21401, 1538)


In [ ]:
Mimic_chart_CV_Formatted_value.head()

In [ ]:
Mimic_chart_MV_Formatted_value.head()

In [ ]:
# [print(i) for i in list(Mimic_chart_CV_Formatted_value.columns) if np.array_equal(Mimic_chart_CV_Formatted_value[i], Mimic_chart_CV_Formatted_value[i].astype(int))==True]
Mimic_chart_CV_Formatted_value.convert_dtypes().dtypes

In [51]:
# partitioning the list of variables in numeric and non-numeric; there are cases where the non-numerics are 
# actually numeric but due to someterms like error or no data they hav gone into non-numeric set.
initial_list_itemid_non_num_CV = []
list_itemid_num_CV = []
for i in Mimic_chart_CV_Formatted_value.columns:
    try :
        pd.to_numeric(Mimic_chart_CV_Formatted_value[i], downcast='float')
        list_itemid_num_CV.append(i)
    except ValueError:
        temp = pd.to_numeric(Mimic_chart_CV_Formatted_value[i].unique(), errors='coerce', downcast='float')
        if np.isnan(temp).sum()!=len(temp):
            list_itemid_num_CV.append(i)
        else:
            initial_list_itemid_non_num_CV.append(i)
        pass
print("Num of numeric variables ", len(list_itemid_num_CV))
print("Initial Num of non-numeric CV varibles ", len(initial_list_itemid_non_num_CV))

Num of numeric variables  1793
Initial Num of non-numeric CV varibles  958


In [52]:
# exploring the non-numeric Carevue era variables and selecting only those that have some categorical forms
ctr_cat = 0 
list_itemid_cat_CV = []  # decided as those that have less than 10 unique values
for i in initial_list_itemid_non_num_CV:
    if len(Mimic_chart_CV_Formatted_value[i].unique()) <= 10:
#         print(" Variable name and id ", item_id_dbsource[item_id_dbsource['itemid']==i]['label'], i)
#         print(Mimic_chart_CV_Formatted_value[i].unique(), "\n")
        list_itemid_cat_CV.append(i)
        ctr_cat = ctr_cat +1
print('Number of categoricals ', len(list_itemid_cat_CV))

Number of categoricals  834


In [53]:
# converting the numeric type variables for CV from string to float
for i in list_itemid_num_CV:
    Mimic_chart_CV_Formatted_value[i] = pd.to_numeric(Mimic_chart_CV_Formatted_value[i], errors='coerce')

In [215]:
# converting the numeric type variables for MV from string to float
list_itemid_num_MV = list(set(list_itemid_num_MV).intersection(set(Mimic_chart_MV_Formatted_value.columns)))
list_itemid_cat_MV = list(set(list_itemid_cat_MV).intersection(set(Mimic_chart_MV_Formatted_value.columns)))
for i in list_itemid_num_MV:
    Mimic_chart_MV_Formatted_value[i] = pd.to_numeric(Mimic_chart_MV_Formatted_value[i], errors='coerce')

In [ ]:
# finding the missing ratio of all the Carevue columns to eliminate the ones that have very high missing ratio
percent_missing_CV = Mimic_chart_CV_Formatted_value.isnull().sum() *100/ len(Mimic_chart_CV_Formatted_value)
Variance_CV_charts = Mimic_chart_CV_Formatted_value[list_itemid_num_CV].var().round(2)
missing_value_df_CV = pd.DataFrame({'column_name': Mimic_chart_CV_Formatted_value.columns,
                                 'percent_missing_CV': percent_missing_CV})
missing_value_df_CV.reset_index(inplace=True)
missing_value_df_CV = pd.merge(item_id_dbsource, missing_value_df_CV, how="inner", on=["itemid"])
missing_value_df_CV.drop(columns=['column_name', 'dbsource'], inplace=True)
missing_value_df_CV.set_index('itemid', inplace=True)
missing_value_df_CV.sort_values(by='percent_missing_CV', ascending=False, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_df_CV)


In [277]:
# finding the missing ratio of all Metavision columns to eliminate the ones that have very high missing ratio
percent_missing_MV = Mimic_chart_MV_Formatted_value.isnull().sum() *100/ len(Mimic_chart_MV_Formatted_value)
missing_value_df_MV = pd.DataFrame({'column_name': Mimic_chart_MV_Formatted_value.columns,
                                 'percent_missing_MV': percent_missing_MV})
missing_value_df_MV.reset_index(inplace=True)
missing_value_df_MV = pd.merge(item_id_dbsource, missing_value_df_MV, how="inner", on=["itemid"])
missing_value_df_MV.drop(columns=['column_name', 'dbsource'], inplace=True)
missing_value_df_MV.set_index('itemid', inplace=True)
missing_value_df_MV.sort_values(by='percent_missing_MV', ascending=False, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(missing_value_df_MV)
    

In [ ]:
missing_value_df_MV[missing_value_df_MV.index.isin(final_itemids_chart_MV_bin)]

In [307]:
# selecting only columns that have less than 80 percent missing values
final_itemids_chart_CV_num = list(missing_value_df_CV[(missing_value_df_CV.index.isin(list_itemid_num_CV)) & (missing_value_df_CV['percent_missing_CV']<80)].index)
final_itemids_chart_CV_cat = list(missing_value_df_CV[(missing_value_df_CV.index.isin(list_itemid_cat_CV)) & (missing_value_df_CV['percent_missing_CV']<80)].index)
# final_itemids_chart_CV_t = final_itemids_chart_CV_num + final_itemids_chart_CV_cat
final_itemids_chart_MV_num = list(missing_value_df_MV[(missing_value_df_MV.index.isin(list_itemid_num_MV)) & (missing_value_df_MV['percent_missing_MV']<80)].index)
final_itemids_chart_MV_cat = list(missing_value_df_MV[(missing_value_df_MV.index.isin(list_itemid_cat_MV)) & (missing_value_df_MV['percent_missing_MV']<80)].index)
# final_itemids_chart_MV_t = final_itemids_chart_MV_num + final_itemids_chart_MV_cat


final_itemids_chart_CV = missing_value_df_CV[(missing_value_df_CV.index.isin(list_itemid_num_CV + list_itemid_cat_CV)) & (missing_value_df_CV['percent_missing_CV']<80)].index
final_itemids_chart_MV = missing_value_df_MV[(missing_value_df_MV.index.isin(list_itemid_num_MV + list_itemid_cat_MV)) & (missing_value_df_MV['percent_missing_MV']<80)].index

In [288]:
# selecting only columns that have less than 80 percent missing values
Mimic_chart_CV_Formatted_value = Mimic_chart_CV_Formatted_value[final_itemids_chart_CV]
Mimic_chart_MV_Formatted_value = Mimic_chart_MV_Formatted_value[final_itemids_chart_MV]

In [289]:
print("CV formatted (ADULTS, less missing) obs shape ",Mimic_chart_CV_Formatted_value.shape)
print("MV formatted (ADULTS, less missing) obs shape ",Mimic_chart_MV_Formatted_value.shape)

CV formatted (ADULTS, less missing) obs shape  (26383, 208)
MV formatted (ADULTS, less missing) obs shape  (21401, 148)


In [ ]:
# checking the dtypes of different columns
Mimic_chart_MV_Formatted_value.dtypes == 'object'

In [291]:
# Final hadm_ids common between lab and chartevents
final_hadmid_CV = list(set(Mimic_chart_CV_Formatted_value.index).intersection(set(Mimic_lab_CV_Formatted_value.index)))
final_hadmid_MV = list(set(Mimic_chart_MV_Formatted_value.index).intersection(set(Mimic_lab_MV_Formatted_value.index)))

In [292]:
# Final dataset Chartevents that has common hadm_ids with labevents dataset
Mimic_chart_CV_Formatted_value = Mimic_chart_CV_Formatted_value.loc[final_hadmid_CV]
Mimic_chart_MV_Formatted_value = Mimic_chart_MV_Formatted_value.loc[final_hadmid_MV]

In [347]:
# Final dataset labevents  that has common hadm_ids with chartevents dataset
Mimic_lab_CV_Formatted_value = Mimic_lab_CV_Formatted_value.loc[final_hadmid_CV]
Mimic_lab_MV_Formatted_value = Mimic_lab_MV_Formatted_value.loc[final_hadmid_MV]

In [348]:
print("Chart CV formatted (ADULTS, less missing, common with labs) obs shape ",Mimic_chart_CV_Formatted_value.shape)
print("Chart MV formatted (ADULTS, less missing, common with labs) obs shape ",Mimic_chart_MV_Formatted_value.shape)

Chart CV formatted (ADULTS, less missing, common with labs) obs shape  (26130, 208)
Chart MV formatted (ADULTS, less missing, common with labs) obs shape  (21125, 148)


In [349]:
print("Lab CV formatted (ADULTS, less missing, common with labs) obs shape ",Mimic_lab_CV_Formatted_value.shape)
print("Lab MV formatted (ADULTS, less missing, common with labs) obs shape ",Mimic_lab_MV_Formatted_value.shape)

Lab CV formatted (ADULTS, less missing, common with labs) obs shape  (26130, 58)
Lab MV formatted (ADULTS, less missing, common with labs) obs shape  (21125, 58)


In [355]:
# Saving the datasets that still need imputation and standardization and one hot encoding too
Mimic_lab_chart_CV.to_csv('Final_MIMIC_lab_chart_CV.csv')
Mimic_lab_chart_MV.to_csv('Final_MIMIC_lab_chart_MV.csv')